In [1]:
import re
import os
import sys
import time
import codecs
import random
import shutil
import smtplib
import requests
import datetime
import openpyxl
import configparser
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from pathlib import Path
from chardet import detect
from functools import reduce
# from datetime import datetime, date, timedelta
# from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
%config Completer.use_jedi = False

In [3]:
def _load_config():
    config_path = "./config.ini"
    with open(config_path, "rb") as ef:
        config_encoding = detect(ef.read())["encoding"]
    config = configparser.ConfigParser()
    config.read_file(codecs.open(config_path, "r", config_encoding))
    return config

In [4]:
config = _load_config()

In [5]:
def _requests_session():
    session = requests.session()
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
    }
    session.headers.update(headers)
    return session

In [6]:
def multiple_replace(sub_dict, text):
     # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, sub_dict.keys())))

     # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: sub_dict[mo.string[mo.start():mo.end()]], text)

In [7]:
def df_m_pick(df_path, col_name):
    df = pd.read_csv(df_path, encoding="utf-8-sig")
    df[df["資料年月"][0]] = df[col_name]
    df_m = df[["代號", "名稱", df["資料年月"][0]]]
    return df_m

In [8]:
# market = ["sii", "otc", "rotc"]
market = ["rotc"]

### 路徑

In [9]:
os.makedirs("./rotc", exist_ok=True)

In [10]:
os.chdir("./rotc")
os.listdir()

['csv', 'xlsx']

In [11]:
# os.chdir("..")
# os.listdir()

In [12]:
# 過去raw csv
past_dir = "./csv/past"

# 過去合併
past_all_dir = "./csv/past_all"


# 全部併成data
data_dir = "./csv/data"
data_rev_dir = "./csv/data/rev"
data_yoy_dir = "./csv/data/yoy"
data_cum_rev_dir = "./csv/data/cum-rev"
data_cum_yoy_dir = "./csv/data/cum-yoy"

# 單天csv
market_dir = "./csv/_market"
# sii_dir = "./csv/_market/sii"
# otc_dir = "./csv/_market/otc"
rotc_dir = "./csv/_market/rotc"

# 單天合併
concat_dir = "./csv/all"

# 當天新增
add_dir = "./csv/add"

# 月營收xlsx
xlsx_dir = "./xlsx"

In [13]:
dir_list = [past_dir, past_all_dir, data_dir, data_rev_dir, data_yoy_dir, data_cum_rev_dir, data_cum_yoy_dir, market_dir, rotc_dir, \
           concat_dir, add_dir, xlsx_dir]

In [14]:
for d in dir_list:
    os.makedirs(d, exist_ok=True)

### 抓以前的資料

In [15]:
Y = list(range(111, 112))
M = list(range(8, 9))
mkt = ["rotc"]

import itertools
cum = len(Y)*len(M)*len(mkt)

os.makedirs("./csv/past", exist_ok=True)

In [16]:
count = 0

for y, m, mk in itertools.product(Y, M, mkt):
    url = "https://mops.twse.com.tw/nas/t21/{}/t21sc03_{}_{}.csv".format(mk, y, m)
    rs = _requests_session()
    res = rs.get(url)
    res.encoding = "utf-8-sig"

    save_path = "./csv/past/m-rev-{}-{}-{}.csv".format(str(y+1911), str(m).zfill(2), mk)
    with open(save_path, "w", encoding="utf-8-sig") as save:
        save.write(res.text.replace("\r\n", "\n"))

    count += 1
    if count % 10 == 0:
        print("{} / {} done.".format(count, cum))
    time.sleep(random.uniform(1.5, 7))
    # print(url)
print("{} / {} done.".format(count, cum))

1 / 1 done.


一堆月營收興櫃合併

In [17]:
def data_date(x):
    match = re.search("(\d+)/(\d+)", x)
    data_Y = int(match[1]) + 1911
    data_m = int(match[2])
    return datetime.datetime(data_Y, data_m, 1) + relativedelta(months=1) - datetime.timedelta(days=1)

In [18]:
os.listdir()

['csv', 'xlsx']

In [19]:
past_list = list(Path("./csv/past").iterdir())
past_list[-1]

WindowsPath('csv/past/m-rev-2022-08-rotc.csv')

In [20]:
past_rotc = list(Path("./csv/past").glob("*otc*"))

In [21]:
len(past_rotc)

80

In [22]:
past_zip = [list(a) for a in zip(past_rotc)]

In [23]:
past_zip[0]

[WindowsPath('csv/past/m-rev-2016-01-rotc.csv')]

In [24]:
def past_concat(df_path):
    dfs = [pd.read_csv(path, encoding="utf-8-sig") for path in df_path]
    df_c = pd.concat(dfs, axis=0, join="outer", ignore_index=True, copy=True)
    df_cols = ["出表日期", "資料年月", "代號", "名稱", "產業", "月營收", "上月營收", "去年當月營收", "MoM", "YoY", "累計營收", "去年累計", "累計YoY", "備註"]
    df_c.columns = df_cols
    drop_cols = ["出表日期", "上月營收", "去年當月營收", "去年累計"]
    df_c.drop(drop_cols, axis=1, inplace=True)
    df_c=df_c.drop_duplicates()
    df_c["資料年月"] = df_c["資料年月"].apply(data_date)
    data_date_str = re.search("\d+-\d+", df_path[0].name)[0]
    concat_path = "{}/m-rev-{}.csv".format(past_all_dir, data_date_str)
    df_c.to_csv(concat_path, index=False, encoding="utf-8-sig")

In [25]:
a = list(map(past_concat, past_zip))

一大堆合併到一張表

In [26]:
past_all = list(Path("./csv/past_all").iterdir())

In [27]:
past_all.sort()

In [28]:
df_t_0_path = past_all[0]
df_t_0_path

WindowsPath('csv/past_all/m-rev-2016-01.csv')

In [29]:
past_all[-1]

WindowsPath('csv/past_all/m-rev-2022-08.csv')

In [30]:
df_t_s = past_all[1:]

In [31]:
def merge_data(dir_name, col_name):
    for file in list(Path("./csv/data/{}".format(dir_name)).iterdir()):
        file.unlink()
        
    df_m_0 = df_m_pick(df_t_0_path, col_name)
    df_m_0_path = "./csv/data/{}/m-rev-data-{}-{}.csv".format(dir_name, dir_name, df_m_0.columns[-1])
    df_m_0.to_csv(df_m_0_path, index=False, encoding="utf-8-sig")
    
    for df_t in df_t_s:
        df_temp_0_path = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
        df_temp_0 = pd.read_csv(df_temp_0_path, encoding="utf-8-sig")
    #     print(df_temp_0_path, df_temp_0.shape)

        df_temp_r = df_m_pick(df_t, col_name)
        df_temp_m = pd.merge(df_temp_0, df_temp_r, on=["代號", "名稱"], how="outer").drop_duplicates()
    #     print(df_t, df_temp_r.shape)

#         df_temp_m = df_temp_m.drop_duplicates()

    #     print(df_temp_m.shape)
        df_temp_m_path = "./csv/data/{}/m-rev-data-{}-{}.csv".format(dir_name, dir_name, df_temp_m.columns[-1])
        df_temp_m.to_csv(df_temp_m_path, index=False, encoding="utf-8-sig")

#         print(df_temp_0.shape, df_temp_r.shape, df_temp_m.shape)
#         print("{} done.\n".format(df_temp_m.columns[-1]))
        
    for file in list(Path("./csv/data/{}".format(dir_name)).iterdir())[:-1]:
        file.unlink()

In [32]:
merge_data("rev", "月營收")
merge_data("yoy", "YoY")
merge_data("cum-rev", "累計營收")
merge_data("cum-yoy", "累計YoY")

去除重複

In [33]:
def data_drop_dup(dir_name):
    path_rev = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
    df = pd.read_csv(path_rev, encoding="utf-8-sig")
    df_group = df.groupby(by=["代號"]).sum()
    df_name_last = df[["代號", "名稱"]].drop_duplicates(subset="代號", keep="last")
    df_data = pd.merge(df_group, df_name_last, on=["代號"], how="outer")
    df_data_cols = df_data.columns[:-1].insert(1, "名稱")
    df_data_fix = df_data[df_data_cols]
    df_data_fix.to_csv(path_rev, index=False, encoding="utf-8-sig")

In [34]:
dir_name_list = ["rev", "yoy", "cum-rev", "cum-yoy"]

In [35]:
for file in dir_name_list:
    data_drop_dup(file)

### 興櫃名單

In [36]:
def data_rotc_update(dir_name):
    path_rev = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
    rotc_link = "https://isin.twse.com.tw/isin/C_public.jsp?strMode=5"
    rs = _requests_session()
    rotc_page = rs.get(rotc_link)
    soup = BS(rotc_page.text, "lxml")
    df_raw = pd.read_html(soup.prettify(), header=0, attrs = {"class": "h4"})[0]
    df_rotc = df_raw[df_raw["市場別"] == "興櫃一般板"].reset_index(drop=True)
    df_rotc["代號"] = df_rotc["有價證券代號及名稱"].str.extract("^(\d{4})").astype("int64")
    df_rotc["名稱"] = df_rotc["有價證券代號及名稱"].str.extract("^\d{4}　(\w+)")
    df_rotc_u = df_rotc[["代號", "名稱"]]
    
    path_rev = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
    df = pd.read_csv(path_rev, encoding="utf-8-sig")
    df_m = df.merge(df_rotc_u, how="right", on="代號", indicator=False)
    df_m["名稱_x"] = df_m["名稱_y"]
    m_cols = df_m.columns.tolist()
    df_m_fix = df_m[m_cols[:-1]]
    m_cols[1] = "名稱"
    df_m_fix.columns = m_cols[:-1]
    df_m_fix.to_csv(path_rev, index=False, encoding="utf-8-sig")

In [37]:
dir_name_list = ["rev", "yoy", "cum-rev", "cum-yoy"]

In [38]:
for file in dir_name_list:
    data_rotc_update(file)